In [71]:
# --- 2020-01 CAU
# --- Natural Language Processing and Information Retrieval
# --- Sentiment Analysis Team Project
# --- 
# --- 

from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from urllib.parse import urlparse, parse_qs
import xlsxwriter
import time

# webdriver dir
browser = webdriver.Chrome('./chromedriver.exe') # watch out the version

In [105]:
# --- Saving results as .xlsx file
def savexlsx(xlsxfilename, array):
    with xlsxwriter.Workbook(xlsxfilename) as workbook:
        worksheet = workbook.add_worksheet()

        # Add a bold format to use to highlight cells
        bold = workbook.add_format({'bold': True})

        for row_num, data in enumerate(array):
            if row_num == 0:
                worksheet.write('A1','title',bold)
                worksheet.write('B1','artist',bold)
                worksheet.write('C1','lyric',bold)
                continue
            worksheet.write_row(row_num, 0, data)
    workbook.close()

In [111]:
for year in range(2006, 2020):
    print("\n=====", year)
    
    # -------------------------------------------------------------------------------------
    # ----- collecting top 100 songs over years ----- 
    chart = []
    songs = 100
    
    chart_url = "https://www.billboard.com/charts/year-end/"+str(year)+"/hot-100-songs"
    browser.get(chart_url)
    
    for rank in range(1, 101): # html code starts with 1 not 0 so,
        try:
            title = browser.find_element_by_xpath("(//div[@class='ye-chart-item__title'])["+str(rank)+"]").text
            artist = browser.find_element_by_xpath("(//div[@class='ye-chart-item__artist'])["+str(rank)+"]").text
            #print("[{}] {} - {}".format(rank, title, artist))
            chart.append([title, artist, ''])
        except:
            print("... song #", rank, " has collecting problem")
            songs -= 1   # sometimes, billboard website doesn't show exactly 100 songs
                         # ex. 2011, there's no #7 song ^^;, 2016 #87
    # -------------------------------------------------------------------------------------
    
    
    # -------------------------------------------------------------------------------------
    # ----------- Search lyrics for songs ----------- 
    ### Be careful! You could be BLOCKED by the website ###
    for rank in range(0, songs):
        org_title = chart[rank][0].replace('/', ' ')         # error_handling (ex.cupid's chokehead/breakfast in america)
        org_artist = chart[rank][1].replace(' Featuring','') # remove featuring artists _error handling
        org_artist = org_artist.split(' (Featuring',1)[0]
        org_artist = org_artist.split(' Or ',1)[0]
        org_artist = org_artist.replace(' Duet With','').replace(' With', '').replace(' X','').replace(' x ',' ')
        org_artist = org_artist.replace('*',' ').replace('#','').replace('/ ','') # these can cause query problem
        print(rank, org_title, org_artist)
        
        browser.get("https://www.musixmatch.com/search/"+org_title+' '+org_artist)
        time.sleep(2) # wait while loading
        
        # Need to check which searched result is matched to original info over webpage but we passed this process
        # because [Best results] song is nearly perfectly same
        """
        title = browser.find_element_by_xpath("(//h2[@class='media-card-title'])[1]")
        artist = browser.find_element_by_xpath("(//h3[@class='media-card-subtitle'])[1]").text.split('feat',1)[0]
        #print(org_title,'-',org_artist, '/', title.text,'-',artist)
        if title.text == org_title and artist == org_artist:
            title.click()
            # ...
        """
        
        try:
            browser.find_element_by_xpath("//a[@class='title']").click() # click Best result
            time.sleep(1)

            # -------------------------------------------------------------------------------------
            # Sometimes, there're some lyrics which are judged as errored or warning by the website
            # But we ignored this notice and collect them no matter what state they are
            # -------------------------------------------------------------------------------------
            # ___case1. lyrics state: OK
            try: 
                lyric = browser.find_element_by_xpath("(//span[@class='lyrics__content__ok'])[1]").text
                # if the content is divided, concatenate them
                try: lyric += browser.find_element_by_xpath("(//span[@class='lyrics__content__ok'])[2]").text
                except: continue
            except:
            # ___case2. lyrics state: Error
                try: 
                    lyric = browser.find_element_by_xpath("(//span[@class='lyrics__content__error'])[1]").text
                    try: lyric += browser.find_element_by_xpath("(//span[@class='lyrics__content__error'])[2]").text
                    except: continue
                except:
            # ___case3. lyrics state: Warning
                    try: 
                        lyric = browser.find_element_by_xpath("(//span[@class='lyrics__content__warning'])[1]").text
                        try: lyric += browser.find_element_by_xpath("(//span[@class='lyrics__content__warning'])[2]").text
                        except:continue
            # ___case 4. lyrics state: Unavailable (ex. restricted)
                    except: continue 
            # -------------------------------------------------------------------------------------
            #print(lyric)
            chart[rank][2] = lyric
            
        except NoSuchElementException: # No searching results ___ this lyrics cell going to be blank
            print("!")
    # -------------------------------------------------------------------------------------
    #print(chart)
    
    savexlsx('./'+str(year)+'_data.xlsx', chart)


===== 2019
0 Old Town Road Lil Nas Billy Ray Cyrus
1 Sunflower (Spider-Man: Into The Spider-Verse) Post Malone & Swae Lee
2 Without Me Halsey
3 Bad Guy Billie Eilish
4 Wow. Post Malone
5 Happier Marshmello & Bastille
6 7 Rings Ariana Grande
7 Talk Khalid
8 Sicko Mode Travis Scott
9 Sucker Jonas Brothers
10 High Hopes Panic! At The Disco
11 Thank U, Next Ariana Grande
12 Truth Hurts Lizzo
13 Dancing With A Stranger Sam Smith & Normani
14 Senorita Shawn Mendes & Camila Cabello
15 I Don't Care Ed Sheeran & Justin Bieber
16 Eastside benny blanco, Halsey & Khalid
17 Going Bad Meek Mill Drake
18 Shallow Lady Gaga & Bradley Cooper
19 Better Khalid
20 No Guidance Chris Brown Drake
21 Girls Like You Maroon 5 Cardi B
22 Sweet But Psycho Ava Max
23 Suge DaBaby
24 Middle Child J. Cole
25 Drip Too Hard Lil Baby & Gunna
26 Someone You Loved Lewis Capaldi
27 Ran$om Lil Tecca
28 If I Can't Have You Shawn Mendes
29 Goodbyes Post Malone Young Thug
30 ZEZE Kodak Black Travis Scott & Offset
31 Better Now

### ------ To do list ------ #
- ? while collecting data, measuring time
- insert missing(errored) lyrics manually
- insert column of songs' duration
 - from ex) https://musicbrainz.org/search?query=for+you+i+will+%28confidence%29&type=recording&method=indexed
